<a href="https://colab.research.google.com/github/emmetorior/CN7030-/blob/main/Multistock_Multifeature_Test_Classifier_from_original_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark pandas numpy

In [81]:
from pyspark.sql import SparkSession
# Importing package
from pyspark.sql.functions import (
    expr, col, lag, lead, window, stddev, mean, first, last,
    when, isnan, count, dayofmonth, month, year, to_date, udf
)
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
import pandas as pd
import numpy as np


In [82]:

####
def load_stock_data(data_dir, pd):
    stock_lookup = {
        "AAPL": "Apple Inc.",
        "NVDA": "NVIDIA Corp",
        "MSFT": "Microsoft",
        "AMZN": "Amazon",
        "QQQ" : "Invesco QQQ Trust",
        "AVGO": "Broadcom Inc.",
        "TSLA": "Tesla Inc.",
        "COST": "Costco Wholesale Corp",
        "NFLX": "Netflix Inc.",
        "GOOGL": "Alphabet Inc."
    }

    df_list = []
    for symbol, name in stock_lookup.items():
      file_path = f"{data_dir}/{symbol}.csv"
      try:
          #df = pd.read_csv(file_path, header=1)  # Skip the first line of each file as it's the header
          #df = pd.read_csv(file_path, usecols=[6])
          df = pd.read_csv(file_path)
          print(f"Loading: {symbol}")
          # new column at index 0 (the first col)
          df.insert(0, "symbol", symbol)
          df_list.append(df)
          combined_df = pd.concat(df_list, ignore_index=True)
          df.reset_index()
      except FileNotFoundError:
            print(f"Oops: {symbol} not found...")

    #combined_df.columns["symbol","date","volume","open","high","low","close","adjclose"]
    #combined_df.columns{"symbol","date","volume","open","high","low","close","adjclose"}
   # combined_df.columns = ["symbol", "date", "volume", "open", "high", "low", "close", "adjclose"]

    return combined_df

#

data_dir = "/content/drive/MyDrive/stockarchive/full_history"
df = load_stock_data(data_dir, pd)
df.head(34000)
#df.show()
 #         df.insert(0, 'symbol', symbol)
 #         df.insert(1, 'date', symbol)
 #         df.insert(2, 'volume', symbol)
 #         df.insert(3, 'open', symbol)
 #         df.insert(4, 'high', symbol)
 #         df.insert(5, 'low', symbol)
 #        df.insert(6, 'close', symbol)
 #        df.insert(7, 'adjclose', symbol)
 #

Loading: AAPL
Loading: NVDA
Loading: MSFT
Loading: AMZN
Loading: QQQ
Loading: AVGO
Loading: TSLA
Loading: COST
Loading: NFLX
Loading: GOOGL


,symbol,date,volume,open,high,low,close,adjclose
0,AAPL,2020-07-02,28484300,367.850006,370.470001,363.640015,364.109985,364.109985
1,AAPL,2020-07-01,27684300,365.119995,367.359985,363.910004,364.109985,364.109985
2,AAPL,2020-06-30,35055800,360.079987,365.980011,360.000000,364.799988,364.799988
3,AAPL,2020-06-29,32661500,353.250000,362.170013,351.279999,361.779999,361.779999
4,AAPL,2020-06-26,51314200,364.410004,365.320007,353.019989,353.630005,353.630005
...,...,...,...,...,...,...,...,...
33995,TSLA,2014-10-13,11268700,238.570007,238.960007,221.000000,224.589996,224.589996
33996,TSLA,2014-10-10,12888300,244.639999,245.889999,235.199997,236.910004,236.910004
33997,TSLA,2014-10-09,7361300,262.250000,265.540009,254.399994,257.010010,257.010010
33998,TSLA,2014-10-08,5055100,260.100006,262.880005,252.639999,259.279999,259.279999


In [83]:

####
def clean_data(df, pd):

    # fix from source how-to-remove-multiple-headers-row-in-a-pandas-data-frame StackOverflow Quang Hoang
    # Is this a problem? Maybe not.
   # df = df[df.ne(df.columns).any(1)]

    ## more pre cleaning later...
    df['date'] = pd.to_datetime(df['date'])
    #df.set_index('date', inplace=True)
    #df.index
    return df
clean_data(df, pd)


,symbol,date,volume,open,high,low,close,adjclose
0,AAPL,2020-07-02,28484300,367.850006,370.470001,363.640015,364.109985,364.109985
1,AAPL,2020-07-01,27684300,365.119995,367.359985,363.910004,364.109985,364.109985
2,AAPL,2020-06-30,35055800,360.079987,365.980011,360.000000,364.799988,364.799988
3,AAPL,2020-06-29,32661500,353.250000,362.170013,351.279999,361.779999,361.779999
4,AAPL,2020-06-26,51314200,364.410004,365.320007,353.019989,353.630005,353.630005
...,...,...,...,...,...,...,...,...
52176,GOOGL,2004-08-25,9188600,52.532532,54.054054,51.991993,53.053055,53.053055
52177,GOOGL,2004-08-24,15247300,55.675674,55.855854,51.836838,52.487488,52.487488
52178,GOOGL,2004-08-23,18256100,55.430431,56.796795,54.579578,54.754753,54.754753
52179,GOOGL,2004-08-20,22834300,50.555557,54.594593,50.300301,54.209209,54.209209


In [113]:
df_qqq = df.loc[ (df['symbol'] == 'QQQ') ]

df_qqq.head()

,symbol,date,volume,open,high,low,close,adjclose
24444,QQQ,2020-07-02,31898400,253.020004,254.270004,251.710007,252.190002,252.190002
24445,QQQ,2020-07-01,34689200,247.639999,251.520004,247.080002,250.490005,250.490005
24446,QQQ,2020-06-30,38909200,242.770004,248.160004,242.580002,247.600006,247.600006
24447,QQQ,2020-06-29,34377800,239.970001,242.850006,237.350006,242.839996,242.839996
24448,QQQ,2020-06-26,52824600,245.779999,245.960007,239.679993,240.220001,240.220001


In [114]:


# Create Spark Session
spark = SparkSession.builder.appName("CN7030 Assn 1 - Stock Price Classifier").getOrCreate()

# Convert Pandas to PySpark
df_qqq = spark.createDataFrame(df_qqq)

windowSpec = Window.partitionBy().orderBy("date").rowsBetween(-8, 0)
df_qqq = df_qqq.withColumn("9_Day_MA", F.avg("close").over(windowSpec))
#df_qqq = df_qqq.withColumn("9_Day_MA", mean("close"))





In [115]:
# 2. Trading Range - daily price range for the stock
df_qqq = df_qqq.withColumn("Trading_Range", col("high") - col("low"))

In [133]:
# 3. Prev Close
df_qqq = df_qqq.withColumn("price_change_percentage", lag("close").over(Window.orderBy("date")))

# 5. price)_change_percentage
#df_qqq = df_qqq.withColumn("prev_close_percentage", col("close") - col("prev_close"))
df_qqq = df_qqq.withColumn("price_change_percentage",
    (col("close") - col("prev_close")) / col("prev_close") * 100
)

In [134]:
# drop nulls
df_qqq = df_qqq.na.drop()
#n = 30
#df.drop(index=df.index[:n], inplace=True)
print(df_qqq)
# Skip some days - I'm skipping
#df = df.iloc[10:]
df_qqq.show()

DataFrame[symbol: string, date: timestamp, volume: bigint, open: double, high: double, low: double, close: double, adjclose: double, 9_Day_MA: double, Trading_Range: double, prev_close: double, prev_close_percentage: double, price_change_percentage: double]
+------+-------------------+--------+----------+---------+----------+---------+------------------+------------------+-------------+----------+---------------------+-----------------------+
|symbol|               date|  volume|      open|     high|       low|    close|          adjclose|          9_Day_MA|Trading_Range|prev_close|prev_close_percentage|price_change_percentage|
+------+-------------------+--------+----------+---------+----------+---------+------------------+------------------+-------------+----------+---------------------+-----------------------+
|   QQQ|1999-03-15 00:00:00| 6369000|   50.4375|  51.5625|  49.90625|     51.5| 44.90494155883789|         50.984375|      1.65625|   50.0625|    2.871410736579276|      2.871

In [120]:
# CHanging to Multi stock version Sample AAPL stock data - this func based on a chunk from google - it only contains the bog standard columns - no citation needed- this function will be deleted - because we'll be reading from the dataset
#def create_sample_stock_data():
#    # Generate a year of simulated stock data
#    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='B')
#    np.random.seed(88)
##FAKE DATA!!
#    data = {
#        'Date': dates,
#        'Volume': np.random.randint(1000000, 5000000, len(dates)),
#        'High': 250 + np.cumsum(np.random.normal(0.1, 1, len(dates))),    # don't worry about the values, they won't be accurate
#        'Low': 200 + np.cumsum(np.random.normal(-0.1, 1, len(dates))),
#        'Close': 145 + np.cumsum(np.random.normal(0, 1, len(dates)))
#    }



In [121]:
# Define an explicit schema to ensure consistent types
#schema = StructTypeOrig([
#    StructField("open", DoubleType(), True),
#    StructField("close", DoubleType(), True),
#    StructField("high", DoubleType(), True),
#    StructField("low", DoubleType(), True),
#    StructField("volume", DoubleType(), True),
#    StructField("price_change_percentage", DoubleType(), True)
#])



In [135]:
# Create label column
df_qqq = df_qqq.withColumn('price_change_label',
    when(col('price_change_percentage') < -2, 'low')
    .when((col('price_change_percentage') >= -2) & (col('price_change_percentage') < 2), 'medium')
    .otherwise('high')
)


In [137]:

# Prepare features
feature_columns = ['open', 'close', 'high', 'low', 'volume']
assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol='features'
)
prepared_df = assembler.transform(df_qqq)

# Index labels
label_indexer = StringIndexer(
    inputCol='price_change_label',
    outputCol='label'
)
prepared_df = label_indexer.fit(prepared_df).transform(prepared_df)

# Split data
(train_data, test_data) = prepared_df.select('features', 'label').randomSplit([0.7, 0.3], seed=42)

# Train the Logistic Regression with max 10 iterations
lr = LogisticRegression(
    maxIter=10,
    regParam=0.3,
    elasticNetParam=0.8,
    labelCol='label',
    featuresCol='features'
)
model = lr.fit(train_data)

# Evaluate the model
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(
    labelCol='label',
    predictionCol='prediction',
    metricName='accuracy'
)
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy}")




Model Accuracy: 0.8221498371335505


In [138]:
# replace original columns here - like in the previous one. I just wrote this app with reduced features to make it easier to create.
prepared_df = prepared_df.withColumn('open', col('open')) \
    .withColumn('close', col('close')) \
    .withColumn('price_change_percentage', col('price_change_percentage')) \
    .withColumn('price_change_label', col('price_change_label'))

# Split into training and test sets
(train_data, test_data) = prepared_df.randomSplit([0.7, 0.3], seed=42)

# Training the log reg
lr = LogisticRegression(
    maxIter=10,
    regParam=0.3,
    elasticNetParam=0.8,
    labelCol='label',
    featuresCol='features'
)
model = lr.fit(train_data)

# predict
predictions = model.transform(test_data)

# Defines the values - change this later, because we want a different type of label...
def map_prediction_to_label(prediction):
    if prediction == 0:
        return 'low'
    elif prediction == 1:
        return 'medium'
    else:
        return 'high'

# map prediction - user defined func ..
map_prediction_udf = udf(map_prediction_to_label, StringType())

# Use the UDF in the DataFrame
results_df = predictions.select(
    'open',
    'close',
    #
    'price_change_percentage',
    'price_change_label',
    # put prediction here?
    'prediction',
    map_prediction_udf(col('prediction')).alias('predicted_label')
)

# Display results
results_df.show()

+----------+----------+-----------------------+------------------+----------+---------------+
|      open|     close|price_change_percentage|price_change_label|prediction|predicted_label|
+----------+----------+-----------------------+------------------+----------+---------------+
|   51.9375|   51.5625|    -0.7220216606498195|            medium|       0.0|            low|
|   50.3125|   48.8125|    -3.5206917850525015|               low|       0.0|            low|
|     51.25|   52.3125|      3.653250773993808|              high|       0.0|            low|
|    51.875|     51.75|    -1.0752688172043012|            medium|       0.0|            low|
|      54.0|   53.5625|      2.084574151280524|              high|       0.0|            low|
|54.4296875|      55.5|     3.6172695449241536|              high|       0.0|            low|
|   55.4375| 55.609375|    0.19707207207207206|            medium|       0.0|            low|
|      54.5|   55.4375|    -0.6718924972004479|            m

In [139]:

# Define the mapping function
def map_prediction_to_label(prediction):
    if prediction == 0:
        return 'low'
    elif prediction == 1:
        return 'medium'
    else:
        return 'high'

# Register the function as a UDF
map_prediction_udf = udf(map_prediction_to_label, StringType())

# Use the UDF in the DataFrame
results_df = predictions.select(
    'open',
    'close',
    'price_change_percentage',
    'price_change_label',
    'prediction',
    map_prediction_udf(col('prediction')).alias('predicted_label')
)

# Display results
results_df.show()

+----------+----------+-----------------------+------------------+----------+---------------+
|      open|     close|price_change_percentage|price_change_label|prediction|predicted_label|
+----------+----------+-----------------------+------------------+----------+---------------+
|   51.9375|   51.5625|    -0.7220216606498195|            medium|       0.0|            low|
|   50.3125|   48.8125|    -3.5206917850525015|               low|       0.0|            low|
|     51.25|   52.3125|      3.653250773993808|              high|       0.0|            low|
|    51.875|     51.75|    -1.0752688172043012|            medium|       0.0|            low|
|      54.0|   53.5625|      2.084574151280524|              high|       0.0|            low|
|54.4296875|      55.5|     3.6172695449241536|              high|       0.0|            low|
|   55.4375| 55.609375|    0.19707207207207206|            medium|       0.0|            low|
|      54.5|   55.4375|    -0.6718924972004479|            m